In [1]:
import pandas as pd 

In [2]:
pd.read_csv('../data/test_acc_timeseries.csv')

,Unnamed: 0,timestamp,signature,net_token_flows,counterparties,type,source,tx_status,direction,counterparty,slot,tx_fee,program_id,PRE_BALANCE,BALANCE,SYMBOL,NAME,MINT,program_name,counterparty_name
0,0,1743941539,6mpqrks4akyjez3jljjubhswnb96oijdfx1dgeznzebhxm...,{'SOL': 1e-09},['5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr'],TRANSFER,SYSTEM_PROGRAM,success,received,5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr,331670303,0.000006,11111111111111111111111111111111,0.017773,0.017773,SOL,Solana,So11111111111111111111111111111111111111111,Unknown Program,Unknown Address
1,0,1743898250,29ap1yrcx2dbjm4mxk8257g5e87by7cgxysds6liaamsxx...,{'SOL': 1e-09},['FLiPgGTXtBtEJoytikaywvWgbz5a56DdHKZU72HSYMFF'],TRANSFER,SYSTEM_PROGRAM,success,received,FLiPgGTXtBtEJoytikaywvWgbz5a56DdHKZU72HSYMFF,331561835,0.000005,11111111111111111111111111111111,0.017773,0.017773,SOL,Solana,So11111111111111111111111111111111111111111,Unknown Program,Unknown Address
2,0,1743898245,2d7d4ndxhzfwz6trn6pzy7pqefkrktqhqjq1zpz5myndvs...,{'SOL': 1e-09},['FLiPGqowc82LLR173hKiFYBq2fCxLZEST5iHbHwj8xKb'],TRANSFER,SYSTEM_PROGRAM,success,received,FLiPGqowc82LLR173hKiFYBq2fCxLZEST5iHbHwj8xKb,331561822,0.000005,11111111111111111111111111111111,0.017773,0.017773,SOL,Solana,So11111111111111111111111111111111111111111,Unknown Program,Unknown Address
3,0,1743896323,nprzwng7slnepcpebwjemkadd6wnuwscrgeweqsnshiznn...,{'SOL': 1e-05},['6UgXZZBoydXRNX6SLjwFDUUgn19GhjBpvNAXU3iepzUV'],TRANSFER,SYSTEM_PROGRAM,success,received,6UgXZZBoydXRNX6SLjwFDUUgn19GhjBpvNAXU3iepzUV,331557009,0.000005,11111111111111111111111111111111,0.017763,0.017773,SOL,Solana,So11111111111111111111111111111111111111111,Unknown Program,Unknown Address
4,0,1743896305,yqsamwung3pc9h5ezjmtr1mudcuuajzye13bmfas2atlvc...,{'SOL': 2e-09},['GUq7PhyAUZko2mPhv3CupmdJKQ61LH8VyrdsRL25q7zg'],TRANSFER,SYSTEM_PROGRAM,success,received,GUq7PhyAUZko2mPhv3CupmdJKQ61LH8VyrdsRL25q7zg,331556964,0.000005,11111111111111111111111111111111,0.017763,0.017763,SOL,Solana,So11111111111111111111111111111111111111111,Unknown Program,Unknown Address
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,0,1699976664,2slwlw7rkrw4xsghxjaxmqidy2yaabjstn6u1h7hekud4d...,{'SOL': 0.0},['FLiPggWYQyKVTULFWMQjAk26JfK5XRCajfyTmD5weaZ7'],TRANSFER,SYSTEM_PROGRAM,success,received,FLiPggWYQyKVTULFWMQjAk26JfK5XRCajfyTmD5weaZ7,229991035,0.000005,11111111111111111111111111111111,NaN,NaN,NaN,NaN,NaN,Unknown Program,Unknown Address
489,0,1699976634,3mwzl67964c2emlndhj37zqyjq6g2vfjqv1asfyhvednzi...,{'SOL': 0.0},['FLiPggWYQyKVTULFWMQjAk26JfK5XRCajfyTmD5weaZ7'],TRANSFER,SYSTEM_PROGRAM,success,received,FLiPggWYQyKVTULFWMQjAk26JfK5XRCajfyTmD5weaZ7,229990970,0.000005,11111111111111111111111111111111,NaN,NaN,NaN,NaN,NaN,Unknown Program,Unknown Address
490,0,1699976603,2iqht33xr4mhcrkvrttzkqgcw16m8usvvv2xhxm2zgcgch...,"{'SOL': -0.236967847, 'So111111111111111111111...",['HzXh3xG4ze9Nw5iBWyUEtGHDXGg6qV2dRtesK5irZsrF...,SWAP,JUPITER,success,received,NaN,229990897,0.000005,TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA,0.469857,0.232884,SOL,Solana,So11111111111111111111111111111111111111111,Unknown Program,Unknown Address
491,0,1699975534,yvctdnezqofcvzv66mkasuuwk3cdz63wshxtshtuafxn52...,"{'SOL': -0.23697041700000002, 'So1111111111111...",['HzXh3xG4ze9Nw5iBWyUEtGHDXGg6qV2dRtesK5irZsrF...,TOKEN_MINT,UNKNOWN,failed,sent,NaN,229988531,0.000005,TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA,0.469862,0.469857,SOL,Solana,So11111111111111111111111111111111111111111,Unknown Program,Unknown Address
